# Import libraries

In [ ]:
!pip install einops timm sentencepiece

In [ ]:
import inspect
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import torch
from transformers import AutoModel, AutoTokenizer
torch.set_grad_enabled(False)

# Instantiate model

In [ ]:
# Instantiate InternLM-XComposer model. via
# https://github.com/InternLM/InternLM-XComposer
model = AutoModel.from_pretrained('internlm/internlm-xcomposer-7b', trust_remote_code=True).cuda().eval()
tokenizer = AutoTokenizer.from_pretrained('internlm/internlm-xcomposer-7b', trust_remote_code=True)
model.tokenizer = tokenizer

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions', exist_ok=True)

# Import datasets

## COVID-19

In [ ]:
constraint22_dataset_covid19_test_captioned_Qwen_VL_Chat = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constrain22_dataset_covid19_test_captioned_Qwen-VL-Chat.csv')
constraint22_dataset_covid19_test_captioned_InternLM_XComposer = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constrain22_dataset_covid19_test_captioned_InternLM-XComposer.csv')
constraint22_dataset_covid19_test_captioned_llava = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constrain22_dataset_covid19_test_captioned_llava-v1.5-13b.csv')
constraint22_dataset_covid19_test_captioned_BLIP_2 = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constrain22_dataset_covid19_test_captioned_BLIP-2.csv')

In [ ]:
constraint22_dataset_covid19_test_captioned_Qwen_VL_Chat = constraint22_dataset_covid19_test_captioned_Qwen_VL_Chat.dropna().reset_index(drop=True)
constraint22_dataset_covid19_test_captioned_InternLM_XComposer = constraint22_dataset_covid19_test_captioned_InternLM_XComposer.dropna().reset_index(drop=True)
constraint22_dataset_covid19_test_captioned_llava = constraint22_dataset_covid19_test_captioned_llava.dropna().reset_index(drop=True)
constraint22_dataset_covid19_test_captioned_BLIP_2 = constraint22_dataset_covid19_test_captioned_BLIP_2.dropna().reset_index(drop=True)

# Set prompts and define a function to call the model

In [ ]:
def prompt_vanilla(entity):
    return inspect.cleandoc(f"""
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
def prompt_with_OCR(entity, OCR):
    return inspect.cleandoc(f"""
    Text on this meme: \"\"\"
    {OCR}
    \"\"\"
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
def prompt_with_caption(entity, caption):
    return inspect.cleandoc(f"""
    Description of this meme: \"\"\"
    {caption}
    \"\"\"
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
# Use InternLM-XComposer for the inference. via
# https://github.com/InternLM/InternLM-XComposer
def get_prediction(image, prompt):
    response = model.generate(prompt, image)
    return response

# Define function to clean responses

In [ ]:
def remap(x):
    x = x.lower()
    if x in {'hero', 'villain', 'victim', 'other'}:
        return x
    elif 'hero' in x and 'villain' not in x and 'victim' not in x and 'other' not in x and 'not hero' not in x and 'not a hero' not in x:
        return 'hero'
    elif 'villain' in x and 'hero' not in x and 'victim' not in x and 'other' not in x and 'not villain' not in x and 'not a villain' not in x:
        return 'villain'
    elif 'victim' in x and 'villain' not in x and 'hero' not in x and 'other' not in x and 'not victim' not in x and 'not a victim' not in x:
        return 'victim'
    elif 'other' in x and 'villain' not in x and 'victim' not in x and 'hero' not in x and 'not other' not in x and 'not an other' not in x:
        return 'other'
    else:
        return None

# Call the `get_prediction` function and save inferences

GROUNDING [ABSENT] & PROMPT [VANILLA]

In [ ]:
covid19_test_grounding_absent_prompt_vanilla = constraint22_dataset_covid19_test_captioned_Qwen_VL_Chat.copy(deep=True)
covid19_test_grounding_absent_prompt_vanilla_images = covid19_test_grounding_absent_prompt_vanilla['image'].values
covid19_test_grounding_absent_prompt_vanilla_entities = covid19_test_grounding_absent_prompt_vanilla['entity'].values
covid19_test_grounding_absent_prompt_vanilla['prediction'] = [get_prediction(image, prompt_vanilla(entity)) for image, entity in zip(covid19_test_grounding_absent_prompt_vanilla_images, covid19_test_grounding_absent_prompt_vanilla_entities)]
covid19_test_grounding_absent_prompt_vanilla.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_covid19_test_grounding[ABSENT]_prompt[VANILLA]_prediction[InternLM-XComposer].csv', index=False)
covid19_test_grounding_absent_prompt_vanilla['prediction'] = covid19_test_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: remap(x))
print(covid19_test_grounding_absent_prompt_vanilla['prediction'].value_counts())
print(covid19_test_grounding_absent_prompt_vanilla['prediction'].isna().sum())
covid19_test_grounding_absent_prompt_vanilla['prediction'] = covid19_test_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(covid19_test_grounding_absent_prompt_vanilla['role'].values, covid19_test_grounding_absent_prompt_vanilla['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(covid19_test_grounding_absent_prompt_vanilla['role'].values, covid19_test_grounding_absent_prompt_vanilla['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

victim     308
other      181
hero       153
villain    116
Name: prediction, dtype: int64
0
0.3230404226998901
              precision    recall  f1-score   support

        hero       0.45      0.37      0.40       189
     villain       0.42      0.26      0.32       190
      victim       0.29      0.47      0.36       189
       other       0.22      0.21      0.21       190

    accuracy                           0.32       758
   macro avg       0.34      0.32      0.32       758
weighted avg       0.34      0.32      0.32       758



GROUNDING [ABSENT] & PROMPT [OCR]

In [ ]:
covid19_test_grounding_absent_prompt_with_OCR = constraint22_dataset_covid19_test_captioned_Qwen_VL_Chat.copy(deep=True)
covid19_test_grounding_absent_prompt_with_OCR_images = covid19_test_grounding_absent_prompt_with_OCR['image'].values
covid19_test_grounding_absent_prompt_with_OCR_entities = covid19_test_grounding_absent_prompt_with_OCR['entity'].values
covid19_test_grounding_absent_prompt_with_OCR_texts = covid19_test_grounding_absent_prompt_with_OCR['OCR'].values
covid19_test_grounding_absent_prompt_with_OCR['prediction'] = [get_prediction(image, prompt_with_OCR(entity, text)) for image, entity, text in zip(covid19_test_grounding_absent_prompt_with_OCR_images, covid19_test_grounding_absent_prompt_with_OCR_entities, covid19_test_grounding_absent_prompt_with_OCR_texts)]
covid19_test_grounding_absent_prompt_with_OCR.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_covid19_test_grounding[ABSENT]_prompt[OCR]_prediction[InternLM-XComposer].csv', index=False)
covid19_test_grounding_absent_prompt_with_OCR['prediction'] = covid19_test_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: remap(x))
print(covid19_test_grounding_absent_prompt_with_OCR['prediction'].value_counts())
print(covid19_test_grounding_absent_prompt_with_OCR['prediction'].isna().sum())
covid19_test_grounding_absent_prompt_with_OCR['prediction'] = covid19_test_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(covid19_test_grounding_absent_prompt_with_OCR['role'].values, covid19_test_grounding_absent_prompt_with_OCR['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(covid19_test_grounding_absent_prompt_with_OCR['role'].values, covid19_test_grounding_absent_prompt_with_OCR['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

victim     597
hero        75
villain     59
other       27
Name: prediction, dtype: int64
0
0.2724011746850207
              precision    recall  f1-score   support

        hero       0.59      0.23      0.33       189
     villain       0.59      0.18      0.28       190
      victim       0.28      0.87      0.42       189
       other       0.22      0.03      0.06       190

    accuracy                           0.33       758
   macro avg       0.42      0.33      0.27       758
weighted avg       0.42      0.33      0.27       758



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [Qwen_VL_Chat]

In [ ]:
covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat = constraint22_dataset_covid19_test_captioned_Qwen_VL_Chat.copy(deep=True)
covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images = covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['image'].values
covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities = covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['entity'].values
covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions = covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['caption'].values
covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images, covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities, covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions)]
covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_covid19_test_grounding[ABSENT]_caption[Qwen-VL-Chat]_prompt[CAPTION]_prediction[InternLM-XComposer].csv', index=False)
covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: remap(x))
print(covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].value_counts())
print(covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].isna().sum())
covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['role'].values, covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['role'].values, covid19_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

victim     310
other      220
villain    183
hero        45
Name: prediction, dtype: int64
0
0.333218262873873
              precision    recall  f1-score   support

        hero       0.62      0.15      0.24       189
     villain       0.43      0.42      0.42       190
      victim       0.32      0.52      0.40       189
       other       0.25      0.29      0.27       190

    accuracy                           0.35       758
   macro avg       0.41      0.35      0.33       758
weighted avg       0.41      0.35      0.33       758



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [InternLM_XComposer]

In [ ]:
covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer = constraint22_dataset_covid19_test_captioned_InternLM_XComposer.copy(deep=True)
covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer_images = covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['image'].values
covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer_entities = covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['entity'].values
covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer_captions = covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['caption'].values
covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer_images, covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer_entities, covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer_captions)]
covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_covid19_test_grounding[ABSENT]_caption[InternLM-XComposer]_prompt[CAPTION]_prediction[InternLM-XComposer].csv', index=False)
covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: remap(x))
print(covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].value_counts())
print(covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].isna().sum())
covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['role'].values, covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['role'].values, covid19_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

victim     376
villain    202
hero        95
other       85
Name: prediction, dtype: int64
0
0.350038323840295
              precision    recall  f1-score   support

        hero       0.53      0.26      0.35       189
     villain       0.47      0.50      0.48       190
      victim       0.31      0.61      0.41       189
       other       0.25      0.11      0.15       190

    accuracy                           0.37       758
   macro avg       0.39      0.37      0.35       758
weighted avg       0.39      0.37      0.35       758



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [llava]

In [ ]:
covid19_test_grounding_absent_prompt_with_caption_llava = constraint22_dataset_covid19_test_captioned_llava.copy(deep=True)
covid19_test_grounding_absent_prompt_with_caption_llava_images = covid19_test_grounding_absent_prompt_with_caption_llava['image'].values
covid19_test_grounding_absent_prompt_with_caption_llava_entities = covid19_test_grounding_absent_prompt_with_caption_llava['entity'].values
covid19_test_grounding_absent_prompt_with_caption_llava_captions = covid19_test_grounding_absent_prompt_with_caption_llava['caption'].values
covid19_test_grounding_absent_prompt_with_caption_llava['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(covid19_test_grounding_absent_prompt_with_caption_llava_images, covid19_test_grounding_absent_prompt_with_caption_llava_entities, covid19_test_grounding_absent_prompt_with_caption_llava_captions)]
covid19_test_grounding_absent_prompt_with_caption_llava.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_covid19_test_grounding[ABSENT]_caption[llava-v1.5-13b]_prompt[CAPTION]_prediction[InternLM-XComposer].csv', index=False)
covid19_test_grounding_absent_prompt_with_caption_llava['prediction'] = covid19_test_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: remap(x))
print(covid19_test_grounding_absent_prompt_with_caption_llava['prediction'].value_counts())
print(covid19_test_grounding_absent_prompt_with_caption_llava['prediction'].isna().sum())
covid19_test_grounding_absent_prompt_with_caption_llava['prediction'] = covid19_test_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(covid19_test_grounding_absent_prompt_with_caption_llava['role'].values, covid19_test_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(covid19_test_grounding_absent_prompt_with_caption_llava['role'].values, covid19_test_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

other      297
victim     251
villain    139
hero        71
Name: prediction, dtype: int64
0
0.3190506536802793
              precision    recall  f1-score   support

        hero       0.49      0.19      0.27       189
     villain       0.37      0.27      0.32       190
      victim       0.31      0.41      0.35       189
       other       0.28      0.44      0.34       190

    accuracy                           0.33       758
   macro avg       0.36      0.33      0.32       758
weighted avg       0.36      0.33      0.32       758



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [BLIP-2]

In [ ]:
covid19_test_grounding_absent_prompt_with_caption_BLIP_2 = constraint22_dataset_covid19_test_captioned_BLIP_2.copy(deep=True)
covid19_test_grounding_absent_prompt_with_caption_BLIP_2_images = covid19_test_grounding_absent_prompt_with_caption_BLIP_2['image'].values
covid19_test_grounding_absent_prompt_with_caption_BLIP_2_entities = covid19_test_grounding_absent_prompt_with_caption_BLIP_2['entity'].values
covid19_test_grounding_absent_prompt_with_caption_BLIP_2_captions = covid19_test_grounding_absent_prompt_with_caption_BLIP_2['caption'].values
covid19_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(covid19_test_grounding_absent_prompt_with_caption_BLIP_2_images, covid19_test_grounding_absent_prompt_with_caption_BLIP_2_entities, covid19_test_grounding_absent_prompt_with_caption_BLIP_2_captions)]
covid19_test_grounding_absent_prompt_with_caption_BLIP_2.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_covid19_test_grounding[ABSENT]_caption[BLIP_2]_prompt[CAPTION]_prediction[InternLM-XComposer].csv', index=False)
covid19_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = covid19_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: remap(x))
print(covid19_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].value_counts())
print(covid19_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].isna().sum())
covid19_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = covid19_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(covid19_test_grounding_absent_prompt_with_caption_BLIP_2['role'].values, covid19_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(covid19_test_grounding_absent_prompt_with_caption_BLIP_2['role'].values, covid19_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

other      352
victim     245
hero        83
villain     78
Name: prediction, dtype: int64
0
0.3049684349361872
              precision    recall  f1-score   support

        hero       0.53      0.23      0.32       189
     villain       0.42      0.17      0.25       190
      victim       0.28      0.37      0.32       189
       other       0.26      0.47      0.33       190

    accuracy                           0.31       758
   macro avg       0.37      0.31      0.30       758
weighted avg       0.37      0.31      0.30       758

